# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:

vac_file = pd.read_csv("../output_data/cities.csv")
vac_file1 = pd.DataFrame(vac_file)
vac_file1


,Unnamed: 0,City,Temperature,Wind,Cloud,Humidity,Latitude,Longitude,Country,Date
0,0,chuy,60.62,5.37,2,83,-33.70,-53.46,UY,1587173981
1,1,hualmay,70.99,6.62,100,78,-11.10,-77.61,PE,1587174162
2,2,mataura,59.00,8.99,35,67,-46.19,168.86,NZ,1587173972
3,3,atuona,82.35,13.06,84,75,-9.80,-139.03,PF,1587173979
4,4,tuatapere,59.00,8.99,34,67,-46.13,167.68,NZ,1587173992
...,...,...,...,...,...,...,...,...,...,...
546,546,lagoa,60.37,11.70,58,72,39.05,-27.98,PT,1587174284
547,547,kurud,96.08,13.18,0,21,20.83,81.72,IN,1587174284
548,548,kailua,84.20,11.41,20,48,21.40,-157.74,US,1587174285
549,549,viedma,73.76,19.80,51,39,-40.81,-63.00,AR,1587174285


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
humidity = vac_file1['Humidity'].astype(float)
# Store 'Lat' and 'Lng' into  locations 
locations = vac_file1[["Latitude", "Longitude"]].astype(float)

fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig.savefig("../output_data/LatitudeVsTemperature.png")

AttributeError: 'Figure' object has no attribute 'savefig'

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
# vac_file1 = pd.DataFrame(vac_file1, columns = ["City", "Temp", "Wind", "Cloud"])
vac_file1 = vac_file1.loc[(vac_file1['Humidity']) >70 & (vac_file1['Humidity'])]
vac_file1 = vac_file1.loc[(vac_file1['Wind']) < 10]
vac_file1 = vac_file1.loc[(vac_file1['Cloud']) == 0]
del vac_file1['Unnamed: 0']
vac_file1 = vac_file1.dropna()
vac_file1

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
hotel_df = pd.DataFrame(vac_file1, columns = ['City', 'Country', 'Latitude', 'Longitude'])
hotel_df['Hotel Name'] = ''
hotel_df['Latitude'][37]

In [ ]:
target_coordinates = f"{hotel_df['Latitude'][37]},{hotel_df['Longitude'][37]}"
target_search = "hotel"
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# run a request using our params dictionary
response = requests.get(base_url, params=params)

# hotel_df.iloc[0][0]
# hotel_df.head()
target_coordinates

In [ ]:
hotel_df.iloc[0][0]

In [ ]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    latitude = row["Latitude"]
    longitude = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{latitude},{longitude}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    
    # print(json.dumps(name_address, indent=4, sort_keys=True))
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [ ]:
hotel_df.head()


In [ ]:
hotel_df['Hotel Name'] = hotel_name
hotel_df.dropna()
hotel_df.to_csv('../output_data/hotel.csv')

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_something = hotel_df['Hotel Name'].tolist()
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=1,
    info_box_content=[f"Bank amount: {hotel}" for hotel in hotel_something]
)

fig.add_layer(hotel_layer)
# Display figure
fig